##### Tzanis Anevlavis and Paulo Tabuada, "A simple hierarchy for computing controlled invariant sets", Submitted to 2020 ACM 23rd International Conference on Hybrid Systems: Computation and Control (HSCC'2020).
##### Section 5: Example 3:
In this notebook, we compute ellipsoidal controlled invariant sets using the method proposed in [LB18].

Then we compare its results with our proposed method for `L=2`.

[LB18] B. Legat, P. Tabuada, R. M. Jungers. Computing controlled invariant sets for hybrid systems with applications to model-predictive control, IFAC-PapersOnLine, Volume 51, Issue 16, 2018, Pages 193-198, ISSN 2405-8963.

The original code of [LB18] can be found in the following [github repository](https://github.com/blegat/SwitchOnSafety.jl).

Example 3 consists of the dynamical system of a truck with N trailers, resulting in a system of dimension `n=2N+1`. 

We record _only_ the time to solve the optimization problem computing the controlled invariant ellispoids. These are the results presented in Table 2. 
For the corresponding volumes of the ellipsoids refer to `paper-examples/example_3.m`, where we have already exported the matrices describing these ellipsoids in order to compute their volumes.

We start with `N=1 (n=3)`.

In [4]:
# Truck with N = 1 trailers (system dimension n = 3):
using MosekTools
using JuMP
using Polyhedra
using MathematicalSystems
using SwitchOnSafety
using LinearAlgebra

solver = with_optimizer(Mosek.Optimizer, QUIET=true)


# Parameters:
# Safe set is a polyhedron:
# safe_set = {x| Gx<=f}
G = [   1.0  0.0  0.0
       -1.0  0.0  0.0
        0.0  1.0  0.0
        0.0  0.0  1.0
        0.0 -1.0  0.0
        0.0  0.0 -1.0]
F = [  0.5
       0.5
      10.0
      10.0
       0.0
       0.0]
safe_set = polyhedron(hrep(G, F), DefaultLibrary{Float64}(solver))

# Input constraints: we work with unconstrained input.
# but seems that ConstrainedLinearControlDiscreteSystem below needs an input set
# therefore, we give a large enough set -- effectively unconstrained input
input_set = polyhedron(convexhull([-1000], [1000]))

# System representation in continuous-time:
A = [ 0.0     1.0    -1.0
     -9.0    -9.2     9.2
      4.5     4.6    -4.6]
B = [0.0, 1.0, 0.0]
# Discretize using Euler-forward method with T=0.4:
A = 0.4*A + I
B = reshape(0.4*B, 3, 1)
# Build system object:
system = ConstrainedLinearControlDiscreteSystem(A, B, safe_set, input_set)

# Begin timing here:
@time begin
#     # In this case, ellipsoids centered at the origin do not work because the origin is on a facet of the safe set.
#     # Therefore, it returns a set with effectively empty interior.
#     
#     # Using ellipsoid centered at the origin:
#     variable = Ellipsoid(symmetric=true)
#     tmp1 = invariant_set(system, solver, variable)
#     ell1 = project(tmp1, 1:3)
    
    # Using ellipsoid given an interior point - use Chebysev center:
    S_procedure_scaling = 1.1884
    cheby_center, cheby_radius = chebyshevcenter(safe_set, solver)
    cheby = [cheby_center; 0.0]
    variable = Ellipsoid(point = SetProg.InteriorPoint(cheby))
    tmp2 = invariant_set(system, solver, variable, λ = S_procedure_scaling)
    ell2 = project(tmp2, 1:3)
end

@show ell2

MOI.get(model, MOI.SolveTime()) = 0.011133193969726562
JuMP.termination_status(model) = OPTIMAL::TerminationStatusCode = 1
JuMP.primal_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.dual_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.objective_value(model) = 0.00040543583397639895
  0.051721 seconds (32.82 k allocations: 2.160 MiB)
ell2 = SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.249986 -0.224602 0.0787268; -0.224602 0.989565 0.772127; 0.0787268 0.772127 1.21755])), [-5.03025e-7, 8.97659, 8.89662])


SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.249986 -0.224602 0.0787268; -0.224602 0.989565 0.772127; 0.0787268 0.772127 1.21755])), [-5.03025e-7, 8.97659, 8.89662])

Continue with `N=2 (n=5)`.

In [5]:
# Truck with N = 2 trailers (system dimension n = 5):
using MosekTools
using JuMP
using Polyhedra
using MathematicalSystems
using SwitchOnSafety
using Plots
using LinearAlgebra

solver = with_optimizer(Mosek.Optimizer, QUIET=true)


# Parameters:
# Safe set is a polyhedron:
# safe_set = {x| Gx<=f}
G = [   1.0     0.0     0.0     0.0     0.0
        0.0     1.0     0.0     0.0     0.0
       -1.0     0.0     0.0     0.0     0.0
        0.0    -1.0     0.0     0.0     0.0
        0.0     0.0     1.0     0.0     0.0
        0.0     0.0     0.0     1.0     0.0
        0.0     0.0     0.0     0.0     1.0
        0.0     0.0    -1.0     0.0     0.0
        0.0     0.0     0.0    -1.0     0.0
        0.0     0.0     0.0     0.0    -1.0]

F = [  0.5
       0.5
       0.5
       0.5
      10.0
      10.0
      10.0
       0.0
       0.0
       0.0]
safe_set = polyhedron(hrep(G, F), DefaultLibrary{Float64}(solver))

# Input constraints: we work with unconstrained input.
# but seems that ConstrainedLinearControlDiscreteSystem below needs an input set
# therefore, we give a large enough set -- effectively unconstrained input
input_set = polyhedron(convexhull([-1000], [1000]))

# System representation in continuous-time:
A = [   0.0    0.0    1.0   -1.0    0.0
        0.0    0.0    0.0    1.0   -1.0
       -9.0    0.0   -9.2    9.2    0.0
        4.5   -4.5    4.6   -9.2    4.6
        0.0    4.5    0.0    4.6   -4.6]
B = [0.0, 0.0, 1.0, 0.0, 0.0]
# Discretize using Euler-forward method with T=0.4:
A = 0.4*A + I
B = reshape(0.4*B, 5, 1)
# Build system object:
system = ConstrainedLinearControlDiscreteSystem(A, B, safe_set, input_set)

# Begin timing here:
@time begin
#     # In this case, ellipsoids centered at the origin do not work because the origin is on a facet of the safe set.
#     # Therefore, it returns a set with effectively empty interior.
#     
#     # Using ellipsoid centered at the origin:
#     variable = Ellipsoid(symmetric=true)
#     tmp1 = invariant_set(system, solver, variable)
#     ell1 = project(tmp1, 1:5)
    
    # Using ellipsoid given an interior point:
    S_procedure_scaling = 1.1884
    cheby_center, cheby_radius = chebyshevcenter(safe_set, solver)
    cheby = [cheby_center; 0.0]
    variable = Ellipsoid(point = SetProg.InteriorPoint(cheby))
    tmp2 = invariant_set(system, solver, variable, λ = S_procedure_scaling)
    ell2 = project(tmp2, 1:5)
end

@show ell2

MOI.get(model, MOI.SolveTime()) = 0.02392292022705078
JuMP.termination_status(model) = OPTIMAL::TerminationStatusCode = 1
JuMP.primal_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.dual_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.objective_value(model) = 0.0001612801769530396
  0.069666 seconds (59.70 k allocations: 4.366 MiB)
ell2 = SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.249881 -0.0636919 -0.0959224 0.164051 0.0970203; -0.0636919 0.249898 -0.0940461 -0.183001 0.0829958; -0.0959224 -0.0940461 0.686048 0.451929 0.183097; 0.164051 -0.183001 0.451929 0.739269 0.573487; 0.0970203 0.0829958 0.183097 0.573487 0.875877])), [-1.15415e-5, -5.37517e-6, 9.17177, 9.10059, 9.06417])


SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.249881 -0.0636919 … 0.164051 0.0970203; -0.0636919 0.249898 … -0.183001 0.0829958; … ; 0.164051 -0.183001 … 0.739269 0.573487; 0.0970203 0.0829958 … 0.573487 0.875877])), [-1.15415e-5, -5.37517e-6, 9.17177, 9.10059, 9.06417])

... and `N=3 (n=7)`.

In [6]:
# Truck with N = 3 trailers (system dimension n = 7):
using MosekTools
using JuMP
using Polyhedra
using MathematicalSystems
using SwitchOnSafety
using Plots
using LinearAlgebra

solver = with_optimizer(Mosek.Optimizer, QUIET=true)


# Parameters:
# Safe set is a polyhedron:
# safe_set = {x| Gx<=f}
G = [    1.0     0.0     0.0     0.0     0.0     0.0     0.0
         0.0     1.0     0.0     0.0     0.0     0.0     0.0
         0.0     0.0     1.0     0.0     0.0     0.0     0.0
        -1.0     0.0     0.0     0.0     0.0     0.0     0.0
         0.0    -1.0     0.0     0.0     0.0     0.0     0.0
         0.0     0.0    -1.0     0.0     0.0     0.0     0.0
         0.0     0.0     0.0     1.0     0.0     0.0     0.0
         0.0     0.0     0.0     0.0     1.0     0.0     0.0
         0.0     0.0     0.0     0.0     0.0     1.0     0.0
         0.0     0.0     0.0     0.0     0.0     0.0     1.0
         0.0     0.0     0.0    -1.0     0.0     0.0     0.0
         0.0     0.0     0.0     0.0    -1.0     0.0     0.0
         0.0     0.0     0.0     0.0     0.0    -1.0     0.0
         0.0     0.0     0.0     0.0     0.0     0.0    -1.0]

F = [   0.5
        0.5
        0.5
        0.5
        0.5
        0.5
       10.0
       10.0
       10.0
       10.0
        0.0
        0.0
        0.0
        0.0]
safe_set = polyhedron(hrep(G, F), DefaultLibrary{Float64}(solver))

# Input constraints: we work with unconstrained input.
# but seems that ConstrainedLinearControlDiscreteSystem below needs an input set
# therefore, we give a large enough set -- effectively unconstrained input
input_set = polyhedron(convexhull([-1000], [1000]))

# System representation in continuous-time:
A = [    0.0         0.0         0.0         1.0          -1.0         0.0         0.0
         0.0         0.0         0.0         0.0           1.0        -1.0         0.0
         0.0         0.0         0.0         0.0           0.0         1.0        -1.0
        -9.0         0.0         0.0        -9.2           9.2         0.0         0.0
         4.5        -4.5         0.0         4.6          -9.2         4.6         0.0
         0.0         4.5        -4.5         0.0           4.6        -9.2         4.6
         0.0         0.0         4.5         0.0           0.0         4.6        -4.6]
B = [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
# Discretize using Euler-forward method with T=0.4:
A = 0.4*A + I
B = reshape(0.4*B, 7, 1)
# Build system object:
system = ConstrainedLinearControlDiscreteSystem(A, B, safe_set, input_set)

# Begin timing here:
@time begin
#     # In this case, ellipsoids centered at the origin do not work because the origin is on a facet of the safe set.
#     # Therefore, it returns a set with effectively empty interior.
#     
#     # Using ellipsoid centered at the origin:
#     variable = Ellipsoid(symmetric=true)
#     tmp1 = invariant_set(system, solver, variable)
#     ell1 = project(tmp1, 1:7)
    
    # Using ellipsoid given an interior point:
    S_procedure_scaling = 1.1884
    cheby_center, cheby_radius = chebyshevcenter(safe_set, solver)
    cheby = [cheby_center; 0.0]
    variable = Ellipsoid(point = SetProg.InteriorPoint(cheby))
    tmp2 = invariant_set(system, solver, variable, λ = S_procedure_scaling)
    ell2 = project(tmp2, 1:7)
end

@show ell2

MOI.get(model, MOI.SolveTime()) = 0.12569594383239746
JuMP.termination_status(model) = SLOW_PROGRESS::TerminationStatusCode = 19
JuMP.primal_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.dual_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.objective_value(model) = 8.301449652469963e-5
  0.171227 seconds (99.21 k allocations: 8.436 MiB)
ell2 = SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.247311 -0.0197818 -0.00201309 -0.111419 0.120498 0.0510998 0.00487735; -0.0197818 0.247695 -0.0804444 -0.0556228 -0.0962673 0.163089 0.0840975; -0.00201309 -0.0804444 0.248017 -0.0403077 -0.0981401 -0.195127 0.0776465; -0.111419 -0.0556228 -0.0403077 0.535786 0.389067 0.200226 0.0508651; 0.120498 -0.0962673 -0.0981401 0.389067 0.62922 0.420648 0.165395; 0.0510998 0.163089 -0.195127 0.200226 0.420648 

SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.247311 -0.0197818 … 0.0510998 0.00487735; -0.0197818 0.247695 … 0.163089 0.0840975; … ; 0.0510998 0.163089 … 0.665933 0.468254; 0.00487735 0.0840975 … 0.468254 0.727407])), [0.000127327, 8.03745e-5, 4.62372e-5, 9.26763, 9.20627, 9.16623, 9.14654])